# 파이토치의 nn.Embedding()  

파이토치에서는 임베딩 벡터를 사용하는 방법이 크게 두가지가 있음.  
바로 임베딩 층(embedding layer)을 만들어 훈련데이터로부터 처음부터 임베딩 벡터를 학습하는 방법과 미리 사전에 훈련된 임베딩 벡터(pre-trained word embedding)들을 가져와 사용하는 방법임.  

# 1. 임베딩 층은 룩업 테이블이다.  

임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어 있어야 함    
어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집 벡터  
임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑하고, 이 밀집 벡터는 인공 신경망의 학습과정에서 가중치가 학습되는 것과 같은 방식으로 훈련됨. 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트 됨. 그리고 밀집 벡터를 임베딩 벡터라고 부름

정수를 밀집 벡터 또는 임베딩 벡터로 맵핑하는것은 어떤 의미일까? 특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있음. 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가짐  
![img](https://wikidocs.net/images/page/33793/lookup_table.PNG)


위의 그림은 단어 great가 정수 인코딩 된 후 테이블로부터 해당 인덱스에 위치한 임베딩 벡터를 꺼내오는 모습을 보여줌. 위의 그림에서는 임베딩 벡터의 차원이 4로 설정 되어져 있음. 그리고 단어 great는 정수 인코딩 과정에서 1,918의 정수로 인코딩 되었고, 그에 따라 단어 집합의 크기만큼의 행을 가지는 테이블에서 인텍스 1,918번에 위치한 행을 단어 great의 임베딩 벡터로 사용함. 이 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 great의 임베딩 벡터 값이 학습됨

파이토치는 단어를 정수 인덱스로 바꾸고, 원-핫 벡터로 한번 더 바꾸고 나서 임베딩 층의 입력으로 사용하는 것이 아니라, 단어를 정수 인덱스로만 바꾼 채로 임베딩 층의 입력으로 사용해도 룩업 테이블이 된 결과인 임베딩 벡터를 리턴함.  
룩업 테이블 과정을 nn.Embedding() 사용하지 않고 구현해봄. 우선 임의의 문장으로부터 단어 집합을 만들고 각 단어에 정수를 부여함.

In [3]:
train_data = 'you need to know how to code'

# 중복을 제거한 단어들의 집합인 단어 집합 생성.
word_set = set(train_data.split())

# 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab = {word: i+2 for i, word in enumerate(word_set)}

vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'how': 2, 'need': 3, 'you': 4, 'to': 5, 'code': 6, 'know': 7, '<unk>': 0, '<pad>': 1}


이제 단어 집합의 크기를 행으로 가지는 임베딩 테이블을 구현함. 단, 여기서 임베딩 벡터의 차원은 3으로 정함

In [5]:
import torch

In [6]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성
embedding_table = torch.FloatTensor([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0],
    [0.2, 0.9, 0.3],
    [0.1, 0.5, 0.7],
    [0.2, 0.1, 0.8],
    [0.4, 0.1, 0.1],
    [0.1, 0.8, 0.9],
    [0.6, 0.1, 0.1],
])

이제 임의의 문장 'you need to run' 에 대해 룩업테이블을 통해 임베딩 벡터들을 가져옴.

In [7]:
# 임의의 샘플 문장
sample = 'you need to run'.split()
idxes = []

# 각 단어를 정수로 변환
for word in sample:
    try:
        idxes.append(vocab[word])
    except KeyError: # 단어 집합에 없는 단어일 경우 <unk>로 대체됨.
        idxes.append(vocab['<unk>'])

idxes = torch.LongTensor(idxes)

# lookup table
# 각 정수를 인덱스로 임베딩 테이블에서 값을 가져옴.
lookup_result = embedding_table[idxes, :]
print(lookup_result)

tensor([[0.2000, 0.1000, 0.8000],
        [0.1000, 0.5000, 0.7000],
        [0.4000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000]])


# 2. 임베딩 층 사용하기

nn.Embedding() 으로 사용할 경우에 우선 전처리는 동일한 과정을 거침

In [8]:
train_data = 'you need to know how to code'
# 중복을 제거한 단어들의 집합인 단어 집합 생성.
word_set = set(train_data.split())

# 단어 집합의 각 단어에 고유한 정수 맵핑.
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

nn.Embedding()을 사용하여 학습 가능한 임베딩 테이블을 만듦

In [10]:
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings = len(vocab),
                               embedding_dim = 3, 
                               padding_idx = 1)

nn.Embedding() 은 크게 두가지 인자를 받는데, 각각 num_embeddings와 embedding_dim임
  - num_embeddings : 임베딩을 할 단어들의 개수. 단어 집합의 크기
  - embedding_dim : 임베딩 할 벡터의 차원. 하이퍼 파라미터
  - padding_idx : 패딩을 위한 토큰의 인덱스 

In [11]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.8033,  1.2248, -0.2897],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.8081, -1.5650,  1.9225],
        [-0.0723,  1.0599, -2.1266],
        [-0.1287, -0.0399,  0.4075],
        [-1.5067, -0.3323, -0.7331],
        [ 1.2240, -0.0720, -0.4234],
        [ 0.2075, -0.4404, -0.8604]], requires_grad=True)
